# Задача 2

In [1]:
from typing import Dict

# Реализация декоратора `cache`

In [3]:
"""
Декоратор для кэширования информации с ограничением на количество запросов.
Аргументы:
- db (str): название базы данных, где кэшируется информация.
- expiration (int): количество раз, когда данные можно взять из кэша.
"""
def cache(db: str, expiration: int):
    cache_storage: Dict[str, Dict] = {}  # Хранилище кэша

    def decorator(func):
        def wrapper(thing: str) -> str:
            # Если элемент уже в кэше
            if thing in cache_storage:
                cache_entry = cache_storage[thing]
                if cache_entry['expires'] > 0:
                    cache_entry['expires'] -= 1
                    return f"Info about: {thing} cached in {db}, expire={cache_entry['expires']}"
                else:
                    del cache_storage[thing]  # Удаляем устаревший кэш
            # Кэшируем новый результат
            result = func(thing)
            cache_storage[thing] = {'data': result, 'expires': expiration}
            return f"{result} from {db}, now cached with expire={expiration}"
        return wrapper
    return decorator

# Определение функции `get_info`

In [4]:
"""
Функция возвращает информацию о предмете (в реальных задачах, возможно, обращение к БД).
"""
@cache(db="postgresql", expiration=5)
def get_info(thing: str) -> str:
    return f"Info about: {thing}"

# Пример использования декоратора и тестирования

In [6]:
# Пример тестирования
def test_cache():
    print(get_info("bike_store"))  # Ожидаемый вывод: Info about: bike_store from postgresql, now cached with expire=5
    print(get_info("bike_store"))  # Ожидаемый вывод: Info about: bike_store cached in postgresql, expire=4
    print(get_info("bike_store"))  # И так далее...
    print(get_info("bike_store"))
    print(get_info("bike_store"))
    print(get_info("bike_store"))  # expire=0, затем кэш обновляется
    print(get_info("bike_store"))  # Кэш обновлен

    # Новый пример с другой базой данных и настройками
    def new_get_info(thing: str) -> str:
        return f"Info about: {thing}"

    new_get_info = cache(db="sqlite", expiration=3)(new_get_info)  # Применяем новый декоратор
    print(new_get_info("bike_store"))  # Ожидаемый вывод: Info about: bike_store from sqlite, now cached with expire=3
    print(new_get_info("bike_store"))
    print(new_get_info("bike_store"))
    print(new_get_info("bike_store"))  # expire=0
    print(new_get_info("bike_store"))  # Кэш обновлен

# Запуск тестов
test_cache()

Info about: bike_store from postgresql, now cached with expire=5
Info about: bike_store cached in postgresql, expire=4
Info about: bike_store cached in postgresql, expire=3
Info about: bike_store cached in postgresql, expire=2
Info about: bike_store cached in postgresql, expire=1
Info about: bike_store cached in postgresql, expire=0
Info about: bike_store from postgresql, now cached with expire=5
Info about: bike_store from sqlite, now cached with expire=3
Info about: bike_store cached in sqlite, expire=2
Info about: bike_store cached in sqlite, expire=1
Info about: bike_store cached in sqlite, expire=0
Info about: bike_store from sqlite, now cached with expire=3
